# earthaccess authentication on a distributed Dask Cluster

In [1]:
import os
import logging
os.environ["HDF5_USE_FILE_LOCKING"] = "FALSE"

import h5netcdf
import h5py
import xarray as xr

import fsspec
import earthaccess

auth = earthaccess.login()


local_cluster = True

# Start a cluster if we don't have one
if 'client' not in locals():
    import dask
    from dask.distributed import Client
    from dask import delayed
    import dask.array as da

    if local_cluster:
        # manual num of workers
        # client = Client(n_workers=8, threads_per_worker=1, silence_logs=logging.ERROR)
        # dask.config.set(scheduler='process')
        client = Client(threads_per_worker=1, silence_logs=logging.ERROR)
    else:
        distributed_cluster_url = "192.168.0.101:8786"
        client = Client(distributed_cluster_url)


for library in (xr, h5netcdf, h5py, fsspec, earthaccess):
    print(f'{library.__name__} v{library.__version__}')

Enter your Earthdata Login username:  earthaccess
Enter your Earthdata password:  ········


xarray v2024.3.0
h5netcdf v1.3.0
h5py v3.11.0
fsspec v2024.3.1.post11+g69f4fe8b
earthaccess v0.9.0.post70.dev0+774aea6


2024-05-06 21:34:25,618 - tornado.application - ERROR - Uncaught exception GET /individual-task-stream/ws (127.0.0.1)
HTTPServerRequest(protocol='http', host='127.0.0.1:8787', method='GET', uri='/individual-task-stream/ws', version='HTTP/1.1', remote_ip='127.0.0.1')
Traceback (most recent call last):
  File "/home/beto/.pyenv/versions/mambaforge/envs/earthaccess-dev/lib/python3.10/site-packages/tornado/web.py", line 1790, in _execute
    result = await result
  File "/home/beto/.pyenv/versions/mambaforge/envs/earthaccess-dev/lib/python3.10/site-packages/tornado/websocket.py", line 273, in get
    await self.ws_connection.accept_connection(self)
  File "/home/beto/.pyenv/versions/mambaforge/envs/earthaccess-dev/lib/python3.10/site-packages/tornado/websocket.py", line 863, in accept_connection
    await self._accept_connection(handler)
  File "/home/beto/.pyenv/versions/mambaforge/envs/earthaccess-dev/lib/python3.10/site-packages/tornado/websocket.py", line 946, in _accept_connection
   

In [2]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 4,Total memory: 31.28 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:34605,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 4
Started: Just now,Total memory: 31.28 GiB
Comm: tcp://127.0.0.1:33129,Total threads: 1
Dashboard: http://127.0.0.1:44705/status,Memory: 7.82 GiB
Nanny: tcp://127.0.0.1:45757,


## Auth on a distributed cluster

Because processes and distributed workers don't share local variables we need a way to pass them the credentials so each local instance of earthaccess can authenticate and open our granules.

This is not optimal and I anticipate that we start embedding the token in the results themselves so earthaccess will grab it from there without us having to manually forward them to the workers.

In [3]:
# this gets executed on each worker
def auth_env(auth):
    os.environ["EARTHDATA_USERNAME"] = str(auth["EARTHDATA_USERNAME"])
    os.environ["EARTHDATA_PASSWORD"] = str(auth["EARTHDATA_PASSWORD"])
    os.environ["HDF5_USE_FILE_LOCKING"] = "FALSE"
    
client.run(auth_env, auth=earthaccess.auth_environ())

{'tcp://127.0.0.1:33129': None,
 'tcp://127.0.0.1:36381': None,
 'tcp://127.0.0.1:38175': None,
 'tcp://127.0.0.1:43501': None}

In [4]:
granules = []
import random
max_samples = 8
# short_name="MUR25-JPL-L4-GLOB-v04.2"
# doi="10.5067/SLREF-CDRV3",

for year in range(2018,2023):
    
    results = earthaccess.search_data(short_name="MUR25-JPL-L4-GLOB-v04.2",
                                      cloud_hosted=True,
                                      # bounding_box=(-66.44,57.54,-63.80,58.27),
                                      temporal=(f"{year}-11", f"{year+1}-02")
                                      )
    if len(results):
        granules.extend(random.sample(results, max_samples))
        
len(granules)

Granules found: 93
Granules found: 93
Granules found: 93
Granules found: 93
Granules found: 92


40

In [5]:
io_params ={
    "fsspec_params": {
        # "skip_instance_cache": True
        "cache_type": "blockcache",  # or "first" with enough space
        "block_size": 1024*1024 # could be bigger
    },
    "h5py_params" : {
        "driver_kwds": { # only recent versions of xarray and h5netcdf allow this correctly
            "page_buf_size": 8*1024*1024, # this one only works in repacked files
            "rdcc_nbytes": 1024*1024 # this one is to read the chunks 
        }

    }
}

# from pqdm.threads import pqdm

# def open_file(link):
#     f_h = fs.open(link, **io_params["fsspec_params"])
#     return f_h

# fileset = pqdm(links, open_file, n_jobs=16, disable=True)


In [6]:
links = [g.data_links()[0] for g in granules]

In [8]:
%%time

# fs = earthaccess.get_fsspec_https_session()

# fo = fs.open(links[0], **io_params["fsspec_params"])

fileset = earthaccess.open(granules)
# ds = xr.open_mfdataset(fileset,
#                        engine="h5netcdf",
#                        lock=False,
#                        data_vars=["analysed_sst"],
#                        compat="override",
#                        coords="minimal",
#                        parallel=True)
# ds
fileset[0]

Opening 40 granules, approx size: 0.07 GB


QUEUEING TASKS | :   0%|          | 0/40 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/40 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/40 [00:00<?, ?it/s]

CPU times: user 630 ms, sys: 161 ms, total: 791 ms
Wall time: 8.09 s


File: <File-like object HTTPFileSystem, https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/MUR25-JPL-L4-GLOB-v04.2/20190124090000-JPL-L4_GHRSST-SSTfnd-MUR25-GLOB-v02.0-fv04.2.nc>
        Cache:<BlockCache:
            block size  :   524288
            block count :   4
            file size   :   1854534
            cache hits  :   0
            cache misses:   0
            total requested bytes: 0>
        

In [9]:
print(str(fileset[0].cache).strip())

<BlockCache:
            block size  :   524288
            block count :   4
            file size   :   1854534
            cache hits  :   0
            cache misses:   0
            total requested bytes: 0>


In [ ]:
fileset[0].read(10)

In [10]:
ds = xr.open_mfdataset(fileset,
                       engine="h5netcdf",
                       parallel=True,
                       compat="override",
                       coords="minimal")
ds

<xarray.Dataset> Size: 1GB
Dimensions:           (time: 40, lat: 720, lon: 1440)
Coordinates:
  * time              (time) datetime64[ns] 320B 2018-11-15T09:00:00 ... 2023...
  * lat               (lat) float32 3kB -89.88 -89.62 -89.38 ... 89.62 89.88
  * lon               (lon) float32 6kB -179.9 -179.6 -179.4 ... 179.6 179.9
Data variables:
    analysed_sst      (time, lat, lon) float64 332MB dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    analysis_error    (time, lat, lon) float64 332MB dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    mask              (time, lat, lon) float32 166MB dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    sea_ice_fraction  (time, lat, lon) float64 332MB dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    sst_anomaly       (time, lat, lon) float64 332MB dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
Attributes: (12/54)
    Conventions:                CF-1.7, ACDD-1.3
    title:                      Daily 0.25-degree MUR SST, Final product
    summary:                    A low-resolution version of the MUR SST analy...
    keywords:                   Oceans > Ocean Temperature > Sea Surface Temp...
    keywords_vocabulary:        NASA Global Change Master Directory (GCMD) Sc...
    standard_name_vocabulary:   NetCDF Climate and Forecast (CF) Metadata Con...
    ...                         ...
    publisher_name:             GHRSST Project Office
    publisher_url:              https://www.ghrsst.org
    publisher_email:            gpc@ghrsst.org
    file_quality_level:         3
    metadata_link:              http://podaac.jpl.nasa.gov/ws/metadata/datase...
    acknowledgment:             Please acknowledge the use of these data with...

In [11]:
sst = ds.analysed_sst.persist()

In [ ]:
dask.visualize(ds.analysed_sst)

In [ ]:
eq_zone = sst.sel(lat=slice(-10, 10), lon=slice(-150, -80))


mzone = eq_zone.groupby('time.year').std(dim='time').compute()
mzone

In [ ]:
import matplotlib.pyplot as plt

df = mzone.mean(dim=["lon","lat"])


ax = df.plot(figsize=(12,4))
ax[0].axes.set_xticks(df.year.values)
ax[0].axes.set_xticklabels(df.year.values)
baseline=0.5
upper_bound=1.0
ax[0].axes.fill_between(df.year, df.values, baseline, where=(df.values >= 0.5), color='orange', alpha=0.4, interpolate=True)
# ax[0].axes.fill_between(df.year, df.values, baseline, where=df.values >= 1.0, color='red', alpha=0.4, interpolate=True)
ax[0].axes.fill_between(df.year, df.values, upper_bound, where=(df.values >= 1.0), color='red', alpha=0.4, interpolate=True)

ax

In [ ]:
computation = sst.groupby('time.year').std(dim='time')
dask.visualize(computation, optimize_graph=True, filename="vis.png")

In [ ]:
%%time
from dask.distributed import performance_report
import gc

with performance_report(filename="dask-pi-earthaccess-smart-2.html"):
    computation.plot.pcolormesh(
        x="lon",
        y="lat",
        col="year",
        col_wrap=4,
        aspect=2,
        robust=True,
        cmap="viridis",
        cbar_kwargs={"label": "Raspberry Pi generated plot"},
    )
client.run(gc.collect)

In [ ]:
dask.visualize(computation, optimize_graph=True, filename="vis.png")

In [ ]:
ds.analysed_sst.std(dim="time").plot()

In [ ]:
import gc

def sstmean_1file(gran_info_single):
    fileobj = fs.open(gran_info_single.data_links()[0], **io_params["fsspec_params"])
    # fileobj = earthaccess.open([gran_info_single])[0]
    data = xr.open_dataset(fileobj)
    mean = data['analysed_sst'].mean().item()
    del fileobj, data
    gc.collect()
    return mean

In [ ]:
# Process several granules in parallel using Dask:
sstmean_1file_parallel = delayed(sstmean_1file)
tasks = [sstmean_1file_parallel(gi) for gi in results]

In [ ]:
dask.visualize(tasks)

In [ ]:
%%time
results = da.compute(*tasks)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 6))
plt.plot( results)
plt.xlabel('Time')
plt.ylabel('Value')
plt.title('Time Series Plot')
plt.show()

In [ ]:
import gc
client.run(gc.collect)